In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp -r /content/drive/MyDrive/pico-llm /content/models

In [ ]:
!git clone https://github.com/prajeshkumarg/pico-llm.git
%cd pico-llm

fatal: destination path 'pico-llm' already exists and is not an empty directory.
/content/pico-llm


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import tiktoken

In [ ]:
def get_word_embedding(word, enc, embed):
    tokens = enc.encode(word)
    t = torch.tensor(tokens, dtype=torch.long)
    vecs = embed(t)
    return vecs.mean(dim=0)
def nearest_tokens(vec, embed, topk=10):
    W = embed.weight
    sims = torch.matmul(W, vec)
    idxs = sims.topk(topk).indices.tolist()
    return idxs

def nearest_words(word, enc, embed, topk=10):
    vec = get_word_embedding(word, enc, embed)
    ids = nearest_tokens(vec, embed, topk)
    return [enc.decode([i]) for i in ids]

In [ ]:
#Load 2 models and compute the clusters of 1st model using nearest_words
def create_embedding_layer(model_location,device="cpu"):
  model = torch.load(model_location,map_location=device)
  num,l = model['token_embed.weight'].shape
  embed = nn.Embedding(num,l)
  embed.weight.data.copy_(model['token_embed.weight'])
  embed.weight.requires_grad = False
  return embed

#Use the clusters to compute cohesion in the next model
def cluster_cohesion(words, enc, embed):
    vecs = torch.stack([F.normalize(get_word_embedding(w, enc, embed), p=2, dim=-1) for w in words])
    sim = F.cosine_similarity(vecs.unsqueeze(1), vecs.unsqueeze(0), dim=2)
    return (sim.sum() - sim.diag().sum()) / (sim.numel() - len(words))


In [ ]:
def extract_token_embeddings(encoder, embedding_layer, save_path=None):
    """
    Extracts tokens and their embeddings from a tiktoken encoder + nn.Embedding layer.

    Args:
        encoder: tiktoken encoding object (e.g. tiktoken.get_encoding("gpt2"))
        embedding_layer: torch.nn.Embedding instance
        save_path: optional path to save as JSON

    Returns:
        A list of dicts: [{ "id": int, "token": str, "embedding": [float, ...] }, ...]
    """
    vocab_size = embedding_layer.num_embeddings
    dim = embedding_layer.embedding_dim
    words = []
    map =[]
    for i in range(vocab_size):
        token_text = encoder.decode([i])
        words.append(token_text)
        map.append({
            "id":"i",
            "token": token_text
        })
    if save_path:
        with open(save_path, "w", encoding="utf-8") as f:
            json.dump(map, f, ensure_ascii=False, indent=2)
        print(f"💾 Saved token embeddings to {save_path}")

    return words

In [ ]:
embed_model1 = create_embedding_layer("/content/models/models/transformer_n2_state.pt")
embed_model2= create_embedding_layer("/content/models/models/transformer_n6_state.pt")


In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")
words1 = extract_token_embeddings(tokenizer,embed_model1)
words2 = extract_token_embeddings(tokenizer,embed_model2)

In [ ]:
clusters1 = {word: nearest_words(word, tokenizer, embed_model1) for word in words1[5000:6000]}

In [ ]:
def find_token_word(word_to_find,words):
  try:
    index = words.index(word_to_find)
    print(f"'{word_to_find}' found at index: {index}")
    return index
  except ValueError:
      print(f"'{word_to_find}' not found in the list.")
      return None



In [ ]:

nearest_words(words1[find_token_word("once",words1)],tokenizer,embed_model1)

'once' found at index: 27078
'once' found at index: 27078


[('once', 27078),
 ('Pers', 30946),
 (' Maine', 15080),
 (' struct', 2878),
 (' cannibal', 39904),
 (' unlikely', 7485),
 (' Wer', 27648),
 (' Bah', 13081),
 ('Finding', 36276),
 (' trauma', 14649)]

In [ ]:
nearest_words(words2[find_token_word("once",words2)],tokenizer,embed_model2)

'once' found at index: 27078


[('once', 27078),
 (' academic', 8233),
 (' appropriation', 34503),
 (' Sword', 11535),
 (' treason', 32204),
 (' Limited', 15302),
 (' punished', 16851),
 (' Ill', 5821),
 (' moratorium', 41772),
 (' Esk', 47534)]

In [ ]:
clusters1[words1[i]]

[(' entirely', 5000),
 (' resurrection', 27556),
 ('shows', 49596),
 (' Literary', 38412),
 (' reforms', 12506),
 (' categorized', 37661),
 (' Breach', 46148),
 ('bomb', 27657),
 ('987', 44183),
 (' blamed', 13772)]

In [ ]:
for i in range(5000,6000):
  print(cluster_cohesion(clusters1[words1[i]], tokenizer, embed_model2))

tensor(0.0006)
tensor(-0.0064)
tensor(0.0114)
tensor(-0.0089)
tensor(0.0232)
tensor(0.0088)
tensor(-0.0184)
tensor(-0.0012)
tensor(0.0043)
tensor(0.0024)
tensor(0.0008)
tensor(-0.0099)
tensor(0.0157)
tensor(-0.0121)
tensor(0.0027)
tensor(-0.0039)
tensor(0.0075)
tensor(-0.0135)
tensor(0.0091)
tensor(-0.0257)
tensor(0.0013)
tensor(0.0150)
tensor(-0.0126)
tensor(0.0052)
tensor(0.0037)
tensor(-0.0005)
tensor(-0.0107)
tensor(-0.0022)
tensor(-0.0082)
tensor(0.0161)
tensor(0.0056)
tensor(0.0030)
tensor(-0.0155)
tensor(0.0124)
tensor(-0.0088)
tensor(0.0232)
tensor(0.0080)
tensor(0.0027)
tensor(-0.0067)
tensor(0.0030)
tensor(0.0102)
tensor(-0.0027)
tensor(0.0098)
tensor(0.0003)
tensor(-0.0022)
tensor(0.0131)
tensor(0.0018)
tensor(0.0021)
tensor(0.0078)
tensor(0.0050)
tensor(0.0110)
tensor(0.0029)
tensor(-0.0099)
tensor(-0.0019)
tensor(0.0032)
tensor(-0.0032)
tensor(-0.0037)
tensor(0.0033)
tensor(-0.0068)
tensor(-0.0004)
tensor(0.0026)
tensor(-0.0091)
tensor(-0.0070)
tensor(-0.0057)
tensor(0.009